In [ ]:
from remote_FT_client import RemoteFTclient
from time import sleep

In [ ]:
client = RemoteFTclient( '192.168.0.103', 10000 )
print( client.prxy.system.listMethods() )

In [ ]:
print( "Get wrist force:" )

for i in range(5):
    print( '\t', client.get_wrist_force() )
    sleep( 0.020 )
    
print( "Bias wrist force" )
client.bias_wrist_force()
sleep( 3.0 )

for i in range(5):
    print( '\t', client.get_wrist_force() )
    sleep( 0.020 )

## echo client

In [4]:
import socket
HOST2 = '192.168.0.103'
PORT2= 65452 
# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = (HOST2, PORT2)
print('connecting to {} port {}'.format(*server_address))
sock.connect(server_address)

try:

    # Send data
    message = b'This is our message. It is very long but will only be transmitted in chunks of 16 at a time'
    print('sending {!r}'.format(message))
    sock.sendall(message)

    # Look for the response
    amount_received = 0
    amount_expected = len(message)

    while amount_received < amount_expected:
        data = sock.recv(16)
        amount_received += len(data)
        print('received {!r}'.format(data))

finally:
    print('closing socket')
    sock.close()

connecting to 192.168.0.103 port 65452
sending b'This is our message. It is very long but will only be transmitted in chunks of 16 at a time'
received b'This is our mess'
received b'age. It is very '
received b'long but will on'
received b'ly be transmitte'
received b'd in chunks of 1'
received b'6 at a time'
closing socket


## test client

In [56]:
import socket
import numpy as np
import struct
from scipy.spatial.transform import Rotation as R
from remote_FT_client import RemoteFTclient
from time import sleep

FTclient = RemoteFTclient( '192.168.0.103', 10000 )
print( FTclient.prxy.system.listMethods() )

def rot2rpy(Rt, degrees=False):
    """ Converts a rotation matrix into rpy / intrinsic xyz euler angle form.
    Args:    Rt (ndarray): The rotation matrix.
    Returns: ndarray: Angles in rpy / intrinsic xyz euler angle form.
    """
    return R.from_matrix(Rt).as_euler('xyz', degrees=degrees)


def rpy2rot(rpy, degrees=False):
    """Converts rpy / intrinsic xyz euler angles into a rotation matrix.
    Args:    rpy (array-like): rpy / intrinsic xyz euler angles.
             degrees (bool) : Whether or not euler_angles are in degrees (True), or radians (False).
    Returns: ndarray: The rotation matrix.
    """
    return R.from_euler('xyz', rpy, degrees=degrees).as_matrix()

HOST2 = '192.168.0.103'
PORT2= 65458
# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = (HOST2, PORT2)
print('connecting to {} port {}'.format(*server_address))
sock.connect(server_address)
print( "Bias wrist force" )
FTclient.bias_wrist_force()
try:
     while True:
        # Send data

        inputstring=0
        #while True:
            #while inputstring!='end':
        inputstring=input("") #Press Enter to continue...
        if (inputstring=='h' or inputstring=='j' or inputstring=='k' or inputstring=='l' or 
            inputstring=='u' or inputstring=='o'or inputstring=='y' or inputstring=='i' or 
            inputstring=='z' or inputstring=='x' or inputstring=='d' or inputstring=='c'or
            inputstring=='home',inputstring=='end'):

            data1=inputstring.encode('ascii')    
            sock.sendall(data1)
        if (inputstring=='end'):
            break
        """
        data = sock.recv(16)
            #amount_received += len(data)
        print('received {!r}'.format(data))
        """

        data = sock.recv(64)  #48 bytes
        #print(data)
        #value = struct.unpack('f',data)
        unpacked = struct.unpack('ffffffffffffffff', data)
        #print(unpacked)
        #print('Received', repr(data))
        TransRotatmatrix=np.zeros([4,4])
        for i in range(4):
            TransRotatmatrix[i][:]=unpacked[0+(4*i):4+(4*i)]
        #print(TransRotatmatrix)
        rpy=rot2rpy(TransRotatmatrix[0:3,0:3],True)
        x=TransRotatmatrix[0][3]*39.3701  #convert to inches 
        y=TransRotatmatrix[1][3]*39.3701
        z=TransRotatmatrix[2][3]*39.3701
        roll=rpy[0]  #In degrees!
        pitch=rpy[1]
        yaw=rpy[2]
        print("x",x,"y",y,"z",z)
        print("roll",roll,"pitch",pitch,"yaw",yaw)

        """"""
        forcesamples=5
        FT_list=[]
        AVG_FT_list=[0]*6  #3 forces 3 torques
        for i in range(5):  #get 5 force samples
            FT_list.append(FTclient.get_wrist_force())
            sleep( 0.020 )            
        for i in range(len(FT_list)): 
            #print("i:",i)
            AVG_FT_list[i]=0
            for j in range(len(FT_list)):  
                AVG_FT_list[i]+=FT_list[j][i]    
            AVG_FT_list[i]=AVG_FT_list[i]/(len(FT_list))
        print("Forces and Torques:")
        print(AVG_FT_list)
        #print( "Bias wrist force" )
        #FTclient.bias_wrist_force()
        #sleep( 3.0 )
        """"""
    
finally:
    
    print('closing socket')
    sock.close()

Connected to http://192.168.0.103:10000
['bias_ft', 'get_ft', 'system.listMethods', 'system.methodHelp', 'system.methodSignature']
connecting to 192.168.0.103 port 65458
Bias wrist force


 u


x -5.445417581295967 y -13.895916417065264 z 9.773463360512256
roll 179.3467723689798 pitch 2.030107685956797 yaw -179.59301568370455
Forces and Torques:
[0.13999999999999999, -0.06000000000000001, -0.16, -0.0024000000000000002, -0.0008, 0]


 l


x -5.446202532652021 y -13.895239411187172 z 8.771782711099089
roll 179.34614273454196 pitch 2.0307321446585553 yaw -179.59956254274005
Forces and Torques:
[0.06000000000000001, -0.02, 0.06000000000000001, -0.020198, 0.0006000000000000001, 0]


...

x -5.446982790726423 y -13.895131465709209 z 8.770037396985293
roll 179.33896779242235 pitch 2.0306079410777698 yaw -179.59412143283876
Forces and Torques:
[0.02, -0.02, 0.76, -0.0088, -0.0024020000000000005, 0]


 j


x -5.446207225933671 y -12.93161012941599 z 8.770846988070012
roll 179.35118157969913 pitch 2.037566610507936 yaw -179.59646015927478
Forces and Torques:
[0.14, -0.9, 0.45999999999999996, 0.038398, 0.014598, 0]


 end


closing socket


In [53]:
a=[0.2, -0.1, 0.1, 0.013, 0.001, -0.007]
b=[0.2, 0.0, 0.5, 0.033, 0.006, -0.008]
c=[0.1, 0.1, 1.3, 0.013, -0.002, -0.007]
d=[0.1, 0.1, 1.2, 0.017, -0.001, -0.008]
e=[0.0, 0.1, 0.4, 0.026, -0.001, -0.00899]

LL=[]
LL.append(a)
LL.append(b)
LL.append(c)
L2=[0]*6
print("LL len",len(LL))
for i in range(len(a)): 
    #print("i:",i)
    L2[i]=0
    for j in range(len(LL)):  
        L2[i]+=LL[j][i]    
    L2[i]=L2[i]/(len(LL))


LL len 3
[0.16666666666666666, 0.0, 0.6333333333333333, 0.019666666666666666, 0.0016666666666666668, -0.007333333333333333]
